<a href="https://colab.research.google.com/github/Kaddouriayoub/PFA/blob/main/Preprocessing%20script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.distance import geodesic

def standardize_coordinates(df):
    """Standardize all latitude/longitude columns to consistent format"""
    coord_cols = [col for col in df.columns if 'lat' in col.lower() or 'lng' in col.lower()]

    for col in coord_cols:
        # Convert to float and handle different decimal formats
        df[col] = pd.to_numeric(df[col], errors='coerce')

        # Standardize longitude to (-180, 180) and latitude to (-90, 90)
        if 'lat' in col.lower():
            df[col] = df[col].apply(lambda x: max(-90, min(90, x)))
        elif 'lng' in col.lower():
            df[col] = df[col].apply(lambda x: ((x + 180) % 360) - 180)  # Normalize to -180..180

        # Round to 6 decimal places (~10cm precision)
        df[col] = df[col].round(6)

    return df

def validate_and_clean(df):
    """Enhanced validation for delivery data"""
    # Standardize coordinates first
    df = standardize_coordinates(df)

    # 1. Calculate delivery duration if not exists
    if 'delivery_duration' not in df.columns:
        if all(col in df.columns for col in ['delivery_time', 'pickup_time']):
            df['delivery_duration'] = (
                pd.to_datetime(df['delivery_time']) - pd.to_datetime(df['pickup_time'])
            ).dt.total_seconds() / 60

    # 2. Validate delivery durations (1min to 24h)
    if 'delivery_duration' in df.columns:
        df = df[(df['delivery_duration'] > 1) & (df['delivery_duration'] < 1440)]

    # 3. Validate GPS coordinates after standardization
    gps_cols = ['lng_pickup', 'lat_pickup', 'lng_delivery', 'lat_delivery']
    if all(col in df.columns for col in gps_cols):
        # Remove rows with invalid coordinates
        df = df[
            (df['lat_pickup'].between(-90, 90)) &
            (df['lng_pickup'].between(-180, 180)) &
            (df['lat_delivery'].between(-90, 90)) &
            (df['lng_delivery'].between(-180, 180)) &
            (df['lat_pickup'].notna()) &
            (df['lng_pickup'].notna()) &
            (df['lat_delivery'].notna()) &
            (df['lng_delivery'].notna())
        ]

    return df

def extract_temporal_features(df, time_cols):
    """Extract hour, day, weekday, and weekend features from timestamps"""
    for col in time_cols:
        if col in df.columns:
            # Ensure datetime type
            df[col] = pd.to_datetime(df[col])

            # Extract features
            df[f'{col}_hour'] = df[col].dt.hour
            df[f'{col}_day'] = df[col].dt.day
            df[f'{col}_weekday'] = df[col].dt.weekday  # Monday=0, Sunday=6
            df[f'{col}_is_weekend'] = df[col].dt.weekday >= 5
            df[f'{col}_month'] = df[col].dt.month
            df[f'{col}_weekofyear'] = df[col].dt.isocalendar().week

    return df

def handle_missing_values(df):
    """Handle missing values with appropriate strategies"""
    gps_cols = [c for c in df.columns if 'gps' in c.lower() or 'lat' in c.lower() or 'lng' in c.lower()]

    for col in df.columns:
        missing_pct = df[col].isnull().mean()

        if missing_pct > 0:
            print(f"{col}: {missing_pct:.1%} missing")

            # Different strategies based on column type and importance
            if missing_pct < 0.05 or col in gps_cols:
                if pd.api.types.is_numeric_dtype(df[col]):
                    df[col] = df[col].fillna(df[col].median())
                else:
                    mode_val = df[col].mode()
                    df[col] = df[col].fillna(mode_val[0] if not mode_val.empty else 'unknown')
            else:
                # Drop columns with too many missing values (except GPS)
                if col not in gps_cols and missing_pct > 0.2:
                    df.drop(col, axis=1, inplace=True)
                    print(f"Dropped column {col} with {missing_pct:.1%} missing values")

    return df

def enhanced_cleaning(pickup_path, delivery_path, output_path):
    """Complete data cleaning pipeline with enhanced features"""
    # Load data
    pickup = pd.read_csv(pickup_path)
    delivery = pd.read_csv(delivery_path)
    print(f"Initial counts - Pickup: {len(pickup):,}, Delivery: {len(delivery):,}")

    # Handle missing values
    pickup = handle_missing_values(pickup)
    delivery = handle_missing_values(delivery)

    # Merge datasets
    merged = pd.merge(
        pickup,
        delivery,
        on='order_id',
        how='inner',
        suffixes=('_pickup', '_delivery')
    )
    print(f"Merged count: {len(merged):,}")

    # Fix timestamp formats (handle year-less dates)
    time_cols = [col for col in merged.columns if 'time' in col.lower()]
    for col in time_cols:
        merged[col] = pd.to_datetime(
            merged[col].astype(str).str.replace(r'(\d+-\d+)', f'{datetime.now().year}-\g<1>', regex=True),
            errors='coerce'
        )

    # Extract temporal features
    merged = extract_temporal_features(merged, time_cols)

    # Standardize and validate coordinates
    merged = standardize_coordinates(merged)

    # Calculate delivery distance if GPS coordinates available
    required_coords = ['lng_pickup', 'lat_pickup', 'lng_delivery', 'lat_delivery']
    if all(col in merged.columns for col in required_coords):
        merged['delivery_distance'] = merged.apply(
            lambda x: geodesic(
                (x['lat_pickup'], x['lng_pickup']),
                (x['lat_delivery'], x['lng_delivery'])
            ).km,
            axis=1
        )

    # Apply validation
    cleaned = validate_and_clean(merged)
    print(f"After validation: {len(cleaned):,}")

    # Select final output columns
    output_cols = [
        # Core identifiers
        'order_id', 'region_id_pickup', 'city_pickup', 'courier_id_pickup',

        # Timestamps and temporal features
        'accept_time_pickup', 'pickup_time', 'delivery_time',
        'accept_time_pickup_hour', 'accept_time_pickup_day', 'accept_time_pickup_weekday',
        'accept_time_pickup_is_weekend', 'accept_time_pickup_month',
        'pickup_time_hour', 'pickup_time_day', 'pickup_time_weekday',
        'pickup_time_is_weekend', 'pickup_time_month',
        'delivery_time_hour', 'delivery_time_day', 'delivery_time_weekday',
        'delivery_time_is_weekend', 'delivery_time_month',

        # Standardized location data
        'lng_pickup', 'lat_pickup', 'lng_delivery', 'lat_delivery',

        # Calculated metrics
        'delivery_duration', 'delivery_distance'
    ]

    # Only keep columns that exist
    output_cols = [col for col in output_cols if col in cleaned.columns]

    cleaned[output_cols].to_csv(output_path, index=False)
    print(f"✅ Successfully saved {len(cleaned):,} records to {output_path}")
    return cleaned

In [42]:
# Run the pipeline
cleaned_data = enhanced_cleaning(
    pickup_path="pickup_hz.csv",
    delivery_path="delivery_hz.csv",
    output_path="cleaned_data_hz.csv"
)

Initial counts - Pickup: 2,130,456, Delivery: 1,861,600
pickup_gps_time: 35.1% missing
pickup_gps_lng: 35.1% missing
pickup_gps_lat: 35.1% missing
accept_gps_time: 51.0% missing
accept_gps_lng: 51.0% missing
accept_gps_lat: 51.0% missing
Merged count: 643,581
After validation: 3,532
✅ Successfully saved 3,532 records to cleaned_data_hz.csv


In [43]:
df = pd.read_csv("cleaned_data_hz.csv")
df.tail()

,order_id,region_id_pickup,city_pickup,courier_id_pickup,accept_time_pickup,pickup_time,delivery_time,accept_time_pickup_hour,accept_time_pickup_day,accept_time_pickup_weekday,...,delivery_time_day,delivery_time_weekday,delivery_time_is_weekend,delivery_time_month,lng_pickup,lat_pickup,lng_delivery,lat_delivery,delivery_duration,delivery_distance
3527,230627,100,Hangzhou,9228,2025-08-04 07:57:00,2025-08-04 10:50:00,2025-08-04 13:03:00,7,4,0,...,4,0,False,8,120.26470,30.38586,119.06446,29.61357,133.0,144.013369
3528,4293658,100,Hangzhou,1050,2025-09-15 11:17:00,2025-09-15 13:45:00,2025-09-16 12:15:00,11,15,0,...,16,1,False,9,120.25696,30.40497,119.70634,29.80145,1350.0,85.396582
3529,2806941,100,Hangzhou,14307,2025-06-17 07:47:00,2025-06-17 08:59:00,2025-06-17 11:37:00,7,17,1,...,17,1,False,6,120.25698,30.40506,120.21481,30.18852,158.0,24.345433
3530,3829039,100,Hangzhou,8691,2025-09-03 11:33:00,2025-09-03 11:52:00,2025-09-03 16:09:00,11,3,2,...,3,2,False,9,120.24717,30.38096,119.91868,30.04970,257.0,48.463642
3531,3898790,100,Hangzhou,13889,2025-10-28 14:52:00,2025-10-28 16:45:00,2025-10-28 18:15:00,14,28,1,...,28,1,False,10,120.24818,30.38054,120.26343,30.32605,90.0,6.216062


In [44]:
# Run the pipeline
cleaned_data = enhanced_cleaning(
    pickup_path="pickup_jl.csv",
    delivery_path="delivery_jl.csv",
    output_path="cleaned_data_jl.csv"
)

Initial counts - Pickup: 195,297, Delivery: 31,415
lng: 0.0% missing
lat: 0.0% missing
aoi_id: 0.0% missing
aoi_type: 0.0% missing
pickup_time: 0.0% missing
pickup_gps_time: 17.5% missing
pickup_gps_lng: 17.5% missing
pickup_gps_lat: 17.5% missing
accept_gps_time: 32.1% missing
accept_gps_lng: 32.1% missing
accept_gps_lat: 32.1% missing
ds: 0.0% missing
accept_gps_lng: 3.0% missing
accept_gps_lat: 3.0% missing
Merged count: 987
After validation: 7
✅ Successfully saved 7 records to cleaned_data_jl.csv


In [46]:
df = pd.read_csv("cleaned_data_jl.csv")
df.tail()

,order_id,region_id_pickup,city_pickup,courier_id_pickup,accept_time_pickup,pickup_time,delivery_time,accept_time_pickup_hour,accept_time_pickup_day,accept_time_pickup_weekday,...,delivery_time_day,delivery_time_weekday,delivery_time_is_weekend,delivery_time_month,lng_pickup,lat_pickup,lng_delivery,lat_delivery,delivery_duration,delivery_distance
2,2671524,74,Jilin,15530,2025-06-04 07:43:00,2025-06-04 09:39:00,2025-06-04 10:36:00,7,4,2,...,4,2,False,6,126.51241,43.85998,126.56652,43.83089,57.0,5.420324
3,72550,85,Jilin,6075,2025-09-07 08:46:00,2025-09-07 16:05:00,2025-09-08 14:53:00,8,7,6,...,8,0,False,9,126.54814,43.87434,126.58476,43.84423,1368.0,4.456461
4,2271103,90,Jilin,10101,2025-06-07 12:41:00,2025-06-07 13:46:00,2025-06-08 13:36:00,12,7,5,...,8,6,True,6,126.57455,43.88741,126.55224,43.90964,1430.0,3.051831
5,4301683,91,Jilin,5591,2025-07-07 09:17:00,2025-07-07 12:48:00,2025-07-08 12:28:00,9,7,0,...,8,1,False,7,126.51884,43.81220,126.56971,43.85205,1420.0,6.028719
6,575979,109,Jilin,13211,2025-08-18 16:00:00,2025-08-18 16:08:00,2025-08-19 11:26:00,16,18,0,...,19,1,False,8,126.55829,43.81944,126.57366,43.82958,1158.0,1.672723


In [47]:
# Run the pipeline
cleaned_data = enhanced_cleaning(
    pickup_path="pickup_sh.csv",
    delivery_path="delivery_sh.csv",
    output_path="cleaned_data_sh.csv"
)

Initial counts - Pickup: 195,698, Delivery: 233,327
pickup_gps_time: 18.8% missing
pickup_gps_lng: 18.8% missing
pickup_gps_lat: 18.8% missing
accept_gps_time: 36.2% missing
accept_gps_lng: 36.2% missing
accept_gps_lat: 36.2% missing
ds: 0.0% missing
region_id: 0.0% missing
city: 0.0% missing
courier_id: 0.0% missing
lng: 0.0% missing
lat: 0.0% missing
aoi_id: 0.0% missing
aoi_type: 0.0% missing
accept_time: 0.0% missing
accept_gps_time: 0.0% missing
accept_gps_lng: 0.0% missing
accept_gps_lat: 0.0% missing
delivery_time: 0.0% missing
delivery_gps_time: 0.0% missing
delivery_gps_lng: 0.0% missing
delivery_gps_lat: 0.0% missing
ds: 0.0% missing
Merged count: 7,411
After validation: 38
✅ Successfully saved 38 records to cleaned_data_sh.csv


In [48]:
df = pd.read_csv("cleaned_data_sh.csv")
df.head()

,order_id,region_id_pickup,city_pickup,courier_id_pickup,accept_time_pickup,pickup_time,delivery_time,accept_time_pickup_hour,accept_time_pickup_day,accept_time_pickup_weekday,...,delivery_time_day,delivery_time_weekday,delivery_time_is_weekend,delivery_time_month,lng_pickup,lat_pickup,lng_delivery,lat_delivery,delivery_duration,delivery_distance
0,2630520,0,Shanghai,822,2025-08-02 07:57:00,2025-08-02 08:59:00,2025-08-02 12:16:00,7,2,5,...,2,5,True,8,121.56738,30.92086,121.51960,31.14838,197.0,25.634276
1,67070,0,Shanghai,14482,2025-09-20 07:47:00,2025-09-20 12:02:00,2025-09-20 16:03:00,7,20,5,...,20,5,True,9,121.52023,30.92064,121.41364,31.23227,241.0,36.016809
2,1735280,5,Shanghai,3021,2025-08-19 13:43:00,2025-08-19 15:02:00,2025-08-20 12:53:00,13,19,1,...,20,2,False,8,121.70018,31.19326,121.45997,30.86469,1311.0,43.046607
3,1163205,5,Shanghai,3564,2025-06-21 07:55:00,2025-06-21 10:24:00,2025-06-21 18:59:00,7,21,5,...,21,5,True,6,121.69476,31.19018,121.40402,31.23175,515.0,28.086334
4,4039479,5,Shanghai,1616,2025-09-21 09:37:00,2025-09-21 18:47:00,2025-09-22 10:19:00,9,21,6,...,22,0,False,9,121.68670,31.16833,121.50416,31.12017,932.0,18.207596


In [49]:
# Run the pipeline
cleaned_data = enhanced_cleaning(
    pickup_path="pickup_yt.csv",
    delivery_path="delivery_yt.csv",
    output_path="cleaned_data_yt.csv"
)

Initial counts - Pickup: 193,828, Delivery: 206,431
region_id: 0.0% missing
city: 0.0% missing
courier_id: 0.0% missing
accept_time: 0.0% missing
time_window_start: 0.0% missing
time_window_end: 0.0% missing
lng: 0.0% missing
lat: 0.0% missing
aoi_id: 0.0% missing
aoi_type: 0.0% missing
pickup_time: 0.0% missing
pickup_gps_time: 24.5% missing
pickup_gps_lng: 24.5% missing
pickup_gps_lat: 24.5% missing
accept_gps_time: 42.9% missing
accept_gps_lng: 42.9% missing
accept_gps_lat: 42.9% missing
ds: 0.0% missing
accept_gps_lng: 1.2% missing
accept_gps_lat: 1.2% missing
Merged count: 6,441
After validation: 40
✅ Successfully saved 40 records to cleaned_data_yt.csv


In [50]:
# Run the pipeline
cleaned_data = enhanced_cleaning(
    pickup_path="pickup_cq.csv",
    delivery_path="delivery_cq.csv",
    output_path="cleaned_data_cq.csv"
)

Initial counts - Pickup: 1,172,703, Delivery: 931,351
pickup_gps_time: 27.5% missing
pickup_gps_lng: 27.5% missing
pickup_gps_lat: 27.5% missing
accept_gps_time: 46.7% missing
accept_gps_lng: 46.7% missing
accept_gps_lat: 46.7% missing
Merged count: 177,378
After validation: 1,003
✅ Successfully saved 1,003 records to cleaned_data_cq.csv


In [51]:
df_city1 = pd.read_csv('cleaned_data_cq.csv')
df_city2 = pd.read_csv('cleaned_data_hz.csv')
df_city3 = pd.read_csv('cleaned_data_sh.csv')
df_city4 = pd.read_csv('cleaned_data_jl.csv')
df_city5 = pd.read_csv('cleaned_data_yt.csv')

In [52]:
merged_df = pd.concat([df_city1, df_city2, df_city3, df_city4, df_city5], ignore_index=True)

In [56]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4620 entries, 0 to 4619
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       4620 non-null   int64  
 1   region_id_pickup               4620 non-null   float64
 2   city_pickup                    4620 non-null   object 
 3   courier_id_pickup              4620 non-null   float64
 4   accept_time_pickup             4620 non-null   object 
 5   pickup_time                    4620 non-null   object 
 6   delivery_time                  4620 non-null   object 
 7   accept_time_pickup_hour        4620 non-null   int64  
 8   accept_time_pickup_day         4620 non-null   int64  
 9   accept_time_pickup_weekday     4620 non-null   int64  
 10  accept_time_pickup_is_weekend  4620 non-null   bool   
 11  accept_time_pickup_month       4620 non-null   int64  
 12  pickup_time_hour               4620 non-null   i

In [57]:
merged_df.to_csv('merged_city_data.csv', index=False)

In [58]:
merged_df = merged_df.drop_duplicates()